<a href="https://colab.research.google.com/github/Kekc/dogsML/blob/main/dogsml/colab/dog_breeds2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar

--2022-07-17 22:32:25--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 756.82M  15.8MB/s    in 62s     

2022-07-17 22:33:27 (12.3 MB/s) - ‘images.tar’ saved [793579520/793579520]



In [2]:
import tarfile
tar_file = tarfile.open("./images.tar", 'r')
tar_file.extractall("tmp/")
tar_file.close()



In [3]:
import os
images_path = "tmp/Images"

for item in os.listdir(images_path):
  os.rename(os.path.join(images_path, item), os.path.join(images_path, item[item.find("-")+1:]))


In [4]:
import os
os.listdir("tmp/Images")

['Greater_Swiss_Mountain_dog',
 'Japanese_spaniel',
 'miniature_poodle',
 'Newfoundland',
 'otterhound',
 'miniature_pinscher',
 'Afghan_hound',
 'Saluki',
 'komondor',
 'Samoyed',
 'giant_schnauzer',
 'bull_mastiff',
 'EntleBucher',
 'miniature_schnauzer',
 'Border_terrier',
 'Irish_terrier',
 'Tibetan_mastiff',
 'basset',
 'toy_terrier',
 'Welsh_springer_spaniel',
 'standard_schnauzer',
 'Brittany_spaniel',
 'bloodhound',
 'Appenzeller',
 'Blenheim_spaniel',
 'Scotch_terrier',
 'Norfolk_terrier',
 'Norwich_terrier',
 'Bernese_mountain_dog',
 'Chihuahua',
 'silky_terrier',
 'keeshond',
 'German_short-haired_pointer',
 'Pekinese',
 'papillon',
 'West_Highland_white_terrier',
 'Tibetan_terrier',
 'vizsla',
 'bluetick',
 'Australian_terrier',
 'Airedale',
 'Brabancon_griffon',
 'toy_poodle',
 'Boston_bull',
 'Siberian_husky',
 'Shih-Tzu',
 'affenpinscher',
 'whippet',
 'Sealyham_terrier',
 'German_shepherd',
 'Mexican_hairless',
 'briard',
 'Norwegian_elkhound',
 'Rhodesian_ridgeback',
 

In [5]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam

In [6]:
img_height = 224
img_width = 224
batch_size = 32
validation_split = 0.2
seed = 123
num_epochs = 1
data_dir = "tmp/Images"

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)
validation_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [8]:
class_names = train_ds.class_names

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [18]:
input_shape = (img_height, img_width, 3)
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
)

data_augmentation = tf.keras.Sequential([
    tfl.RandomFlip("horizontal"),
    tfl.RandomRotation(0.2),
    tfl.RandomContrast(factor=(0, 0.2)),
    tfl.RandomHeight(factor=(0, 0.3)),
    tfl.RandomWidth(factor=(0, 0.3)),
    tfl.RandomTranslation(
        height_factor=0.3,
        width_factor=0.3,
        fill_mode="nearest",
    ),
    tfl.RandomZoom(
        height_factor=(0, 0.3),
        width_factor=(0, 0.3),
        fill_mode="nearest",
    ),
])

base_model.trainable = False
inputs = tf.keras.Input(shape=input_shape)

x_data = data_augmentation(inputs)
# x_data = inputs
# x_data = tf.keras.applications.mobilenet_v2.preprocess_input(x_data)

x_data = base_model(x_data, training=False)
x_data = tfl.GlobalAveragePooling2D()(x_data)
# x_data = tfl.Dropout(0.2)(x_data)
outputs = tfl.Dense(
    len(class_names),
    activation="softmax",
    kernel_initializer=glorot_uniform(seed=0)
)(x_data)

transfer_model = tf.keras.Model(inputs, outputs)

In [19]:
base_learning_rate = 1 * 10**-3
transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [20]:
num_epochs = 5
history_top = transfer_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=num_epochs,
)

base_model = transfer_model.layers[2]
base_model.trainable = True
fine_tune_at = 190
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

transfer_model.compile(
    optimizer=Adam(learning_rate=base_learning_rate/10),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

history_base = transfer_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=num_epochs * 2,
    initial_epoch=history_top.epoch[-1],
)

Epoch 1/5
515/515 [==============================] - 46s 81ms/step - loss: 2.5272 - accuracy: 0.3884 - val_loss: 0.9457 - val_accuracy: 0.7123
Epoch 2/5
515/515 [==============================] - 42s 80ms/step - loss: 1.5907 - accuracy: 0.5678 - val_loss: 0.8120 - val_accuracy: 0.7483
Epoch 3/5
515/515 [==============================] - 41s 79ms/step - loss: 1.4013 - accuracy: 0.6057 - val_loss: 0.7766 - val_accuracy: 0.7607
Epoch 4/5
515/515 [==============================] - 42s 81ms/step - loss: 1.3014 - accuracy: 0.6296 - val_loss: 0.7824 - val_accuracy: 0.7646
Epoch 5/5
515/515 [==============================] - 41s 79ms/step - loss: 1.2355 - accuracy: 0.6471 - val_loss: 0.7741 - val_accuracy: 0.7736
Epoch 5/10
515/515 [==============================] - 54s 96ms/step - loss: 0.9789 - accuracy: 0.7129 - val_loss: 0.7575 - val_accuracy: 0.7726
Epoch 6/10
515/515 [==============================] - 50s 96ms/step - loss: 0.8385 - accuracy: 0.7512 - val_loss: 0.7884 - val_accuracy: 0.76

In [21]:
transfer_model.save("dog_breed_colab.h5", save_format="h5")


# New Section